In [1]:
import os
import warnings
import pandas as pd

In [2]:
def loadRes(problem, sizes, methods, new=False, limit60=False):
    # init a list to store processed results
    results = []
    # loop through each size
    for s in sizes:
        for m in methods:
            # retrive metrics
            metrics = retriveData(problem, s, m, new, limit60)
            # append the results to the list
            results.append(metrics)
    # convert into a Pandas DataFrame
    result_df = pd.DataFrame(results)
    # reshape
    result_df = reshapeDataframe(result_df, problem, sizes)
    return result_df

def retriveData(problem, size, method, new, limit60):
    # method map
    method_mapping = {
    "cls": "RC", "cls-p":"RC-P", "thd": "LT", "thd-p": "LT-P",
    "exact": "EX",  "rel": "RR", "root": "N1", "lrn": "RL", "ste": "RS",
    "cls50": "RC", "thd50": "LT", "lrn50": "RL", "ste50": "RS",
    "cls100": "RC", "thd100": "LT", "lrn100": "RL", "ste100": "RS",
    "cls200": "RC", "thd200": "LT", "lrn200": "RL", "ste200": "RS",
    "cls300": "RC", "thd300": "LT", "lrn300": "RL", "ste300": "RS"
    }
    # dir and file
    data_path = "./result/"
    if problem == "rb":
        if method[-2:] == "-p":
            csv_file = os.path.join(data_path, f"{problem}_{method[:-2]}100_{size}-p.csv")
        else:
            csv_file = os.path.join(data_path, f"{problem}_{method}_{size}.csv")
    else:
        if new and method[-2:] == "-p":
            csv_file = os.path.join(data_path, f"{problem}_{method[:-2]}100_{size}-{size}-p_new.csv")
        elif new:
            csv_file = os.path.join(data_path, f"{problem}_{method}_{size}-{size}_new.csv")
        elif method[-2:] == "-p":
            csv_file = os.path.join(data_path, f"{problem}_{method[:-2]}100_{size}-{size}-p.csv")
        else:
            csv_file = os.path.join(data_path, f"{problem}_{method}_{size}-{size}.csv")
    #print(csv_file)
    if not os.path.exists(csv_file):
        # output a warning
        warnings.warn(f"File not found: {csv_file}")
        # return None values for all metrics
        return {
            "Method": method_mapping[method],
            "Problem Size": size,
            "Obj Mean": None,
            "Obj Median": None,
            "% Infeasible": None,
            "% Unsolved": None,
            "Time (Sec)": None,
            "Viol Mean": None,
            "Viol Max": None,
            "Num Viol Mean": None
        }
    # load data
    df = pd.read_csv(csv_file)
    # calculate the required metrics
    obj_mean = df["Obj Val"].mean()  # mean of the objective values
    obj_median = df["Obj Val"].median()  # median of the objective values
    if "Num Violations" in df.columns:
        percent_infeasible = (df["Num Violations"] > 0).mean() * 100  # percentage of infeasible instances
    else:
        percent_infeasible = (df["Constraints Viol"] > 0).mean() * 100  # percentage of infeasible instances
    #mean_viols = df["Mean Violation"].mean()  # average mean violation
    #max_viols = df["Max Violation"].max()  # maximum violation
    #num_viols = df["Num Violations"].mean()  # average number of violations
    percent_unsolved = df["Obj Val"].isna().mean() * 100  # percentage of unsolved cases
    time_mean = df["Elapsed Time"].mean()  # Mean of the elapsed solving times
    # append the results to the list
    metrics = {
        "Method": method_mapping[method],
        "Problem Size": size,
        "Obj Mean": obj_mean,
        "Obj Median": obj_median,
        "% Infeasible": percent_infeasible,
        "% Unsolved": percent_unsolved,
        "Time (Sec)": time_mean,
        #"Viol Mean": mean_viols,
        #"Viol Max": max_viols,
        #"Num Viol Mean": num_viols
    }
    return metrics

def reshapeDataframe(df, problem, sizes):
    # define the metrics to include
    metrics = ["Obj Mean", "Obj Median", "% Infeasible", "% Unsolved", "Time (Sec)"]#, "Viol Mean", "Viol Max", "Num Viol Mean"]
    # define the method to include
    methods = ["RC", "RC-P", "LT", "LT-P", "EX", "RR", "N1", "RL", "RS"]
    # reshape
    reshaped_data = []
    for method in methods:
        for metric in metrics:
            row = {"Method": method, "Metric": metric}
            for s in sizes:
                value = df[(df["Method"] == method) & (df["Problem Size"] == s)][metric].values
                # check if the value has exactly one element
                if len(value) != 1:
                    raise ValueError(f"Unexpected number of values ({len(value)}) for Method: {method}, Metric: {metric}, Problem Size: {s}")
                if problem == "rb":
                    row[f"{s*2}×4"] = value[0]
                else:
                    row[f"{s}×{s}"] = value[0]
            reshaped_data.append(row)
    # convert to a DataFrame
    reshaped_df = pd.DataFrame(reshaped_data)
    return reshaped_df

### Convex Quadratic

In [3]:
sizes = [5, 10, 20, 50, 100, 200, 500, 1000]
methods = ["cls", "cls-p", "thd", "thd-p", "exact", "rel", "root", "lrn", "ste"]
loadRes(problem="cq", sizes=sizes, methods=methods)

,Method,Metric,5×5,10×10,20×20,50×50,100×100,200×200,500×500,1000×1000
0,RC,Obj Mean,0.620135,-1.595961e+00,-4.237328e+00,-1.219593e+01,-1.354054e+01,-31.617419,-73.312205,-142.675594
1,RC,Obj Median,0.428045,-1.891001e+00,-4.307350e+00,-1.219962e+01,-1.359898e+01,-31.706233,-73.376950,-142.726241
2,RC,% Infeasible,4.000000,8.000000e+00,1.000000e+00,1.000000e+00,4.000000e+00,3.000000,14.000000,18.000000
3,RC,% Unsolved,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
4,RC,Time (Sec),0.001853,1.865625e-03,1.856163e-03,1.868107e-03,2.159805e-03,0.002132,0.002530,0.004238
5,RC-P,Obj Mean,0.621961,-1.600231e+00,-4.237511e+00,-1.219571e+01,-1.353867e+01,-31.616601,-73.311309,-142.679793
6,RC-P,Obj Median,0.428045,-1.891001e+00,-4.307350e+00,-1.219962e+01,-1.357044e+01,-31.706233,-73.376950,-142.733843
7,RC-P,% Infeasible,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
8,RC-P,% Unsolved,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
9,RC-P,Time (Sec),0.008629,1.374018e-02,4.515936e-03,5.469356e-03,5.000036e-03,0.005039,0.006525,0.009040


### Simple Non-Convex

In [4]:
sizes = [5, 10, 20, 50, 100, 200, 500, 1000]
methods = ["cls100", "cls-p", "thd100", "thd-p", "exact", "rel", "root", "lrn100", "ste100"]
loadRes(problem="nc", sizes=sizes, methods=methods, new=True, limit60=False)

,Method,Metric,5×5,10×10,20×20,50×50,100×100,200×200,500×500,1000×1000
0,RC,Obj Mean,0.382745,1.101563,0.227695,7.712910e-01,1.663565,1.471528,0.526173,1.422244
1,RC,Obj Median,0.239879,0.837723,0.216590,7.519165e-01,1.594214,1.436150,0.526173,0.809124
2,RC,% Infeasible,1.000000,3.000000,0.000000,2.000000e+00,0.000000,1.000000,4.000000,3.000000
3,RC,% Unsolved,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
4,RC,Time (Sec),0.001919,0.001936,0.001928,2.014108e-03,0.002193,0.002170,0.002871,0.003962
5,RC-P,Obj Mean,0.383347,1.104406,0.227695,7.720791e-01,1.663565,1.471307,0.523836,1.423367
6,RC-P,Obj Median,0.239879,0.852273,0.216590,7.519165e-01,1.594214,1.436150,0.526173,0.809124
7,RC-P,% Infeasible,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
8,RC-P,% Unsolved,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
9,RC-P,Time (Sec),0.009967,0.006731,0.004506,5.773771e-03,0.006032,0.005363,0.006108,0.011546


### Rosenbrock

In [5]:
sizes = [1, 10, 100, 1000, 10000]
methods = ["cls100", "cls-p", "thd100", "thd-p", "exact", "rel", "root", "lrn100", "ste100"]
loadRes(problem="rb", sizes=sizes, methods=methods, limit60=False)

,Method,Metric,2×4,20×4,200×4,2000×4,20000×4
0,RC,Obj Mean,23.265538,59.393365,5.035068e+02,5.938372e+03,6.688320e+04
1,RC,Obj Median,21.475520,48.857842,4.617061e+02,5.792519e+03,6.679711e+04
2,RC,% Infeasible,3.000000,0.000000,1.000000e+00,1.000000e+00,2.400000e+01
3,RC,% Unsolved,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
4,RC,Time (Sec),0.001926,0.001944,2.087076e-03,3.257055e-03,1.213258e-02
5,RC-P,Obj Mean,23.499273,59.393365,5.041639e+02,5.942178e+03,9.815323e+04
6,RC-P,Obj Median,21.475520,48.857842,4.617061e+02,5.792519e+03,7.272989e+04
7,RC-P,% Infeasible,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
8,RC-P,% Unsolved,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00
9,RC-P,Time (Sec),0.006242,0.004775,5.157626e-03,7.005303e-03,8.244643e-02
